# DLO-JZ Optimisation de l'apprentissage - Jour 1 

Optimisation système d'une boucle d'apprentissage *Resnet-152*.

![car](./images/optimisation.png)

## Objet du notebook

Le but de ce *notebook* est d'optimiser un code d'apprentissage d'un modèle *Resnet-50* sur *Imagenet* pour Jean Zay en implémentant :
* **TP 1** : l'accélération GPU
* **TP 2** : l'*Automatic Mixed Precision*
* **TP 3** : le *Channels Last Memory Format*
* **TP 4** : le *Profiler*

Les cellules dans ce *notebook* ne sont pas prévues pour être modifiées, sauf rares exceptions indiquées dans les commentaires. Les TP se feront en modifiant les codes `dlojz1_X.py`.

Les directives de modification seront marquées par l'étiquette **TODO** dans le *notebook* suivant.
 
Les solutions sont présentes dans le répertoire `solutions/`.

*Notebook rédigé par l'équipe assistance IA de l'IDRIS, juin 2023*

------------------------

### Environnement de calcul

Les fonctions *python* de gestion de queue SLURM dévelopées par l'IDRIS et les fonctions dédiées à la formation DLO-JZ sont à importer.

Le module d'environnement pour les *jobs* et la taille des images sont fixés pour ce *notebook*.

**TODO :** choisir un *pseudonyme* (maximum 5 caractères) pour vous différencier dans la queue SLURM et dans les outils collaboratifs pendant la formation et la compétition.

In [ ]:
from idr_pytools import display_slurm_queue, gpu_jobs_submitter, search_log
from dlojz_tools import controle_technique, compare, GPU_underthehood, plot_accuracy, lrfind_plot, imagenet_starter
MODULE = 'pytorch-gpu/py3/2.4.0'
image_size = 224
account = 'for@a100'
name = 'pseudo'   ## Pseudonyme à choisir

Création d'un répertoire `checkpoints/` si cela n'a pas déjà été fait.

In [ ]:
!mkdir -p checkpoints

------------------------------------

### Gestion de la queue SLURM

Pour afficher vos jobs dans la queue SLURM :

In [ ]:
display_slurm_queue(name)

**Remarque**: Cette fonction est utilisée plusieurs fois dans ce *notebook*. Elle permet d'afficher la queue de manière dynamique, rafraichie toutes les 5 secondes. Elle ne s'arrête que lorsque la queue est vide. Si vous désirez reprendre la main sur le *notebook*, il vous suffira d'arrêter manuellement la cellule avec le bouton *stop*. Cela a bien sûr aucun impact les jobs soumis.

Si vous voulez retirer TOUS vos jobs de la queue SLURM, décommenter et exécuter la cellule suivante :


In [ ]:
#!scancel -u $USER

Si vous voulez retirer UN de vos jobs de la queue SLURM, décommenter, compléter et exécuter la cellule suivante :

In [ ]:
#!scancel <jobid>

--------------

### Différence entre deux scripts

Pour comparer son code avec les solutions mises à disposition, la fonction suivante permet d'afficher une page HTML contenant un différentiel de fichiers texte.

In [ ]:
s1 = "./dlojz1_1.py"
s2 = "./solutions/dlojz1_1.py"
compare(s1, s2)

Voir le résultat du différentiel de fichiers sur la page suivante (attention au spoil !) :

[compare.html](compare.html)

------------------

## Dataset et modèle

Cette partie permet de visualiser les caractéristiques du *dataset* et du modèle utilisés.

### Imagenet

#### Train set

In [ ]:
import os
import torchvision
import torchvision.transforms as transforms
import torch
import numpy as np
import matplotlib.pyplot as plt

transform = transforms.Compose([ 
        transforms.RandomResizedCrop(224),             # Random resize - Data Augmentation
        transforms.RandomHorizontalFlip(),              # Horizontal Flip - Data Augmentation
        transforms.ToTensor(),                          # convert the PIL Image to a tensor
        transforms.Normalize(mean=(0.485, 0.456, 0.406),
                             std=(0.229, 0.224, 0.225))
        ])
    
    
train_dataset = torchvision.datasets.ImageNet(root=os.environ['ALL_CCFRSCRATCH']+'/imagenet',
                                                  transform=transform)
train_dataset

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,    
                                           batch_size=4,
                                           shuffle=True)
batch = next(iter(train_loader))
print('X train batch, shape: {}, data type: {}, Memory usage: {} bytes'
      .format(batch[0].shape, batch[0].dtype, batch[0].element_size()*batch[0].nelement()))
print('Y train batch, shape: {}, data type: {}, Memory usage: {} bytes'
      .format(batch[1].shape, batch[1].dtype, batch[1].element_size()*batch[1].nelement()))

img = batch[0][0].numpy().transpose((1,2,0))
plt.imshow(img)
plt.axis('off')
labels_cls, labels_id = torch.load(os.environ['ALL_CCFRSCRATCH']+'/imagenet/meta.bin')
label = labels_cls[np.unique(labels_id)[batch[1][0].numpy()]]
_ = plt.title('label class: {}'.format(label[0]))

#### Validation set

In [ ]:
val_transform = transforms.Compose([
                                    transforms.Resize((256, 256)),
                                    transforms.CenterCrop(224),
                                    transforms.ToTensor(),   # convert the PIL Image to a tensor
                                    transforms.Normalize(mean=(0.485, 0.456, 0.406),
                                    std=(0.229, 0.224, 0.225))])

val_dataset = torchvision.datasets.ImageNet(root=os.environ['ALL_CCFRSCRATCH']+'/imagenet', split='val',
                        transform=val_transform)
val_dataset

### Resnet-152

In [ ]:
import torchvision.models as models
model = models.resnet152()
print('number of total parameters: {}'.format(sum([p.numel() for p in model.parameters()])))
print('number of trainable parameters: {}'.format(sum([p.numel() for p in model.parameters() if p.requires_grad])))

--------------


# TP1_0 : Baseline CPU

Ce TP consiste à appliquer le code *baseline* pour prendre en main les fonctionnalités de test et découvrir le code.

**TODO :**
1. Exécuter les cellules suivantes (le *job* prend plus de 10 minutes environ)
2. Puis, ouvrir le fichier [dlojz1_0.py](dlojz1_0.py)

Remarque :
* l'option *test* lance un apprentissage de 50 itérations.
* les chronomètres mesurent les temps de la 2e à la 50e itération et restitue un temps moyen par itération.
* les parties `DON'T MODIFY` dans le script ne doivent pas être modifiées.

In [ ]:
n_gpu = 1
batch_size = 128

In [ ]:
command = f'./dlojz1_0.py -b {batch_size} --image-size {image_size} --test --no-pin-memory --test-nsteps 10'
command

Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment-là**.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.

Copier-coller la sortie `jobid = ['xxxxx']` dans la cellule suivante.

Puis, rebasculer la cellule précédente en mode `Raw NBConvert`, afin d'eviter de relancer un job par erreur.

In [ ]:
#jobid = ['159506']

In [ ]:
display_slurm_queue(name)

### Quizz

L'éxécution étant assez longue (~ 10 min.), un quizz vous attend : [Quizz TP1_0](https://www.deepmama.com/quizz/dlojz_quizz1.html)

In [ ]:
controle_technique(jobid)

Le code *baseline* `dlojz1_0.py` a été exécuté sur le CPU (contrairement à ce qui est indiqué par le contrôle technique) en mode *test*, soit sur 50 itérations.

Dans le prochain exercice nous verrons ensemble l'accélération sur 1 GPU.


![Garage](images/stop.png "Arrêtez-vous ici! Une présentation vous attend avant le prochain TP.")

--------------------------------

# TP1_1 : Accélération GPU

Voir la [documentation pytorch](https://pytorch.org/docs/stable/generated/torch.Tensor.to.html)

**TODO** : dans le script [dlojz1_1.py](./dlojz1_1.py):
* Définir la variable `gpu` et envoyer le modèle dans la mémoire du GPU.

* Envoyer toutes les *metrics* dans la mémoire du GPU.

* Envoyer les *batches* d'images d'entrée et les *labels* associés sur le GPU, pour **les étapes de *training* et de *validation***.

In [ ]:
n_gpu = 1
batch_size = 128
command = f'./dlojz1_1.py -b {batch_size} --image-size {image_size} --test'
command

Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment-là**.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.

Copier-coller la sortie `jobid = ['xxxxx']` dans la cellule suivante.

Puis, rebasculer la cellule précédente en mode `Raw NBConvert`, afin d'eviter de relancer un job par erreur.

In [ ]:
#jobid = ['159564']

In [ ]:
display_slurm_queue(name)

### Quizz

L'éxécution étant assez longue, un quizz vous attend : [Quizz TP1_1](https://www.deepmama.com/quizz/dlojz_quizz2.html)

In [ ]:
controle_technique(jobid)

### Test d'occupation mémoire

Afin de mesurer l'impact de la taille de batch sur l'occupation mémoire et sur le *throughput*, la cellule suivante permet de soumettre plusieurs *jobs* avec des tailles de *batch* croissantes. Dans les cas où la mémoire est saturée et dépasse la capacité du GPU, le système renverra une erreur *CUDA Out of Memory*.

Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment-là**.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.

In [ ]:
n_gpu = 1
batch_size = [8, 16, 32, 64, 128, 256, 512]
command = [f'./dlojz1_1.py -b {b} --image-size {image_size} --test'
          for b in batch_size]
jobids = gpu_jobs_submitter(command, n_gpu, MODULE, name=name,
                   account=account, time_max='00:10:00')
print(f'jobids = {jobids}')

Copier-coller la sortie `jobids = ['xxxxx', ...]` dans la cellule suivante.

Puis, rebasculer la cellule précédente en mode `Raw NBConvert`, afin d'eviter de relancer un job par erreur.

In [ ]:
#jobids = ['902357', '902358', '902359', '902360', '902361', '902362', '902363', '902365']

In [ ]:
display_slurm_queue(name)

In [ ]:
GPU_underthehood(jobids)

Le dernier *job* a atteint le seuil *CUDA Out Of Memory* :

In [ ]:
controle_technique([jobids[-1]])

![Garage](images/stop.png "Arrêtez-vous ici! Une présentation vous attend avant le prochain TP.")

-------------------------------------

# TP1_2 : Automatic Mixed Precision

Voir la [documentation de l'IDRIS](http://www.idris.fr/ia/mixed-precision.html#en_pytorch)

**TODO** : dans le script [dlojz1_2.py](./dlojz1_2.py):
* Importer les fonctionnalités liées à l'*Automatic Mixed Precision*.

* Initialiser le *scaler*.  

* Implémenter l'*autocasting* (le changement de précision, FP32 à FP16) dans le *forward* , avec la ligne `with autocast():` dans la boucle de *training* **et** la boucle de validation.

* Implémenter le *gradient scaling* pour la seule boucle de *training*. **Note**: À la place des lignes `loss.backward()` et `optimizer.step()`.


In [ ]:
n_gpu = 1
batch_size = 128
command = f'./dlojz1_2.py -b {batch_size} --image-size {image_size} --test'
command

Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment-là**.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.

Copier-coller la sortie `jobid = ['xxxxx']` dans la cellule suivante.

Puis, rebasculer la cellule précédente en mode `Raw NBConvert`, afin d'eviter de relancer un job par erreur.

In [ ]:
#jobid = ['902431']

In [ ]:
display_slurm_queue(name)

### Quizz

L'éxécution étant assez longue, un quizz vous attend : [Quizz TP1_2](https://www.deepmama.com/quizz/dlojz_quizz3.html)

In [ ]:
controle_technique(jobid)

### Test d'occupation mémoire

Afin de mesurer l'impact de la taille de batch sur l'occupation mémoire et sur le *throughput*, la cellule suivante permet de soumettre plusieurs *jobs* avec des tailles de *batch* croissantes. Dans les cas où la mémoire est saturée et dépasse la capacité du GPU, le système renverra une erreur *CUDA Out of Memory*.

Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment-là**.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.

Copier-coller la sortie `jobids = ['xxxxx', ...]` dans la cellule suivante.

Puis, rebasculer la cellule précédente en mode `Raw NBConvert`, afin d'eviter de relancer un job par erreur.

In [ ]:
#jobids = ['903148', '903150', '903151', '903152', '903154', '903155', '903156', '903157']

In [ ]:
display_slurm_queue(name)

In [ ]:
GPU_underthehood(jobids)

### Changement de taille de batch

**TODO :** Choisir pour la suite du TP une taille de batch par GPU qui vous semble la plus pertinente selon le test précédent.

In [ ]:
## Choisir un batch size optimal
bs_optim = 512

In [ ]:
n_gpu = 1
command = f'./dlojz1_2.py -b {bs_optim} --image-size {image_size} --test'
command

Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment-là**.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.

Copier-coller la sortie `jobids = ['xxxxx', ...]` dans la cellule suivante.

Puis, rebasculer la cellule précédente en mode `Raw NBConvert`, afin d'eviter de relancer un job par erreur.

In [ ]:
#jobid = ['903167']

In [ ]:
display_slurm_queue(name)

In [ ]:
controle_technique(jobid)

![Commentaires](images/cedez.png "Assurez-vous que tout se passe bien avant de continuer!")

-----------------------------------------

# TP1_3 : Channels Last Memory Format

Voir la [documentation pytorch](https://pytorch.org/tutorials/intermediate/memory_format_tutorial.html)

**TODO** : dans le script [dlojz1_3.py](./dlojz1_3.py):
* Lors de l'envoie du modèle au GPU, configurer le paramètre `memory_format` avec l'option *Channel Last Memory*.

* Lors de l'envoie des images d'entrée au GPU, configurer le paramètre `memory_format` avec l'option *Channel Last Memory*.


In [ ]:
n_gpu = 1
command = f'./dlojz1_3.py -b {bs_optim} --image-size {image_size} --test'
command

Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment-là**.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.

Copier-coller la sortie `jobids = ['xxxxx', ...]` dans la cellule suivante.

Puis, rebasculer la cellule précédente en mode `Raw NBConvert`, afin d'eviter de relancer un job par erreur.

In [ ]:
#jobid = ['902659']

In [ ]:
display_slurm_queue(name)

In [ ]:
controle_technique(jobid)

### Test d'occupation mémoire

Afin de mesurer l'impact de la taille de batch sur l'occupation mémoire et sur le *throughput*, la cellule suivante permet de soumettre plusieurs *jobs* avec des tailles de *batch* croissantes. Dans les cas où la mémoire est saturée et dépasse la capacité du GPU, le système renverra une erreur *CUDA Out of Memory*.

Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment-là**.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.

Copier-coller la sortie `jobids = ['xxxxx', ...]` dans la cellule suivante.

Puis, rebasculer la cellule précédente en mode `Raw NBConvert`, afin d'eviter de relancer un job par erreur.

In [ ]:
#jobids = ['2069429', '2069430', '2069431', '2069432', '2069433', '2069435', '2069437']

In [ ]:
display_slurm_queue(name)

In [ ]:
GPU_underthehood(jobids)

![Garage](images/stop.png "Arrêtez-vous ici! Une présentation vous attend avant le prochain TP.")

----------------------
## Garage - Mise à niveau

On fixe le *batch size* et la taille d'image pour ce TP.

In [ ]:
image_size = 224

# TP1_4 : Profiler

### Implémentation du profiler PyTorch

Voir la [documentation de l'IDRIS](http://www.idris.fr/jean-zay/pre-post/profiler_pt.html).

**TODO** : dans le script [dlojz1_4.py](./dlojz1_4.py) :

* Importer les librairies liées au *profiler* PyTorch.

* Configurer le *profiler* et ses paramètres.

```python
    # pytorch profiler setup
	prof =  profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
                    schedule=schedule(wait=1, warmup=1, active=5, repeat=1),
                    on_trace_ready=tensorboard_trace_handler('./profiler/' + os.environ['SLURM_JOB_NAME'] 
                                               + '_' + os.environ['SLURM_JOBID'] + '_bs' +
                                               str(mini_batch_size)  + '_is' + str(args.image_size)),
                    profile_memory=True,
                    record_shapes=False, 
                    with_stack=False,
                    with_flops=False
                    )
```

* Englober toute la boucle d'apprentissage (validation comprise) dans le *context* `prof`.


* Indiquer au *profiler* la fin de chaque itération d'apprentissage (avant la validation).


* Ajouter des balises comme suit :

```python
    #TODO tag forward step with record functions
    with record_function("optimizer zero grad"): optimizer.zero_grad()
    # Implement autocasting
    with autocast():
        with record_function("inference"): outputs = model(images)
        with record_function("loss function"): loss = criterion(outputs, labels)
    
    if args.test: chrono.backward()       

    #TODO tag backward step with record functions
    # Implement gradient scaling
    with record_function("gradient compute"): scaler.scale(loss).backward()
    with record_function("optimizer step and weights update"): scaler.step(optimizer)
    with record_function("Scaler update"): scaler.update()
```

### Génération d'une trace profiler
Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment-là**.

__Remarques__ : 
* le profilage sera actif sur 5 *steps* donc nous n'exécutons l'entraînement que sur 8 steps grâce à l'argument `--test-nsteps 8`.
* Nous lancerons deux *jobs* de prise de trace, avec une taille de *batch* de `512` puis une taille de *batch* de `64`.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.

Puis, rebasculer la cellule précédente en mode `Raw NBConvert`, afin d'eviter de relancer un job par erreur.

In [ ]:
#jobid = ['205798', '205799']

In [ ]:
display_slurm_queue(name)

In [ ]:
controle_technique(jobid[0])

**TODO** : vérifier que 2 traces ont bien été générées dans les répertoires `profiler/<name>_<jobid>_bs512_is224/` et `profiler/<name>_<jobid>_bs64_is224/` et  sous la forme d'un fichier `.json`:

In [ ]:
!tree profiler/

### Visualisation des traces profiler avec TensorBoard <a id="visu_tensorboard_gpu"></a>

**TODO** : visualiser cette trace grâce à l'application TensorBoard en suivant les étapes suivantes :
* ouvrir [jupyterhub.idris.fr](https://jupyterhub.idris.fr) dans un nouvel onglet du navigateur
* ouvrir une nouvelle instance JupyterHub en cliquant sur **Add New JupyterLab Instance**
* sélectionner **Spawn server on SLURM node** (on va réserver un GPU)
* sélectionner **Tensorboard** dans le menu **Frontend**
* définir le chemin des logs **$WORK/DLO-JZ/Jour1/tp_dlojz_jour1/profiler** dans **TensorBoard logs directory**

<div><img src="images/slurm_spawner_a100_tensorboard.png" width="550"></div>


* sélectionner l'option avancée `--partition=Octo-GPU A100 SXM4 with 80 GB GPU mem`

<div><img src="images/slurm_spawner_a100_tensorboard_advanced.png" width="550"></div>

* lancer l'instance TensorBoard
<div><img src="images/slurm_spawner_a100_tensorboard_start.png" width="550"></div>

__Remarque__ : le premier démarrage de TensorBoard peut prendre un peu de temps. Il faut parfois faire preuve d'un peu de patience lorsqu'on utilise cet outil mais ça en vaut la peine :)

Vous disposez de **2 traces** : une avec un *batch_size* de **512** et l'autre avec un *batch_size* de **64**.

**TODO** : en naviguant dans les différents onglets du TensorBoard, chercher à répondre aux questions suivantes :
* Comparer les 2 traces : le GPU est-il bien utilisé ? (mémoire max utilisée, *occupancy*, *efficiency*)
* les *TensorCores* sont-ils bien sollicités ? Quelles sont les couches éligibles aux *TensorCores*? (Voir les vue *Operator* et *GPU Kernel*)
* essayer de repérer les grandes étapes de calcul sur la *timeline* de l'exécution (onglet *Trace*)

**IMPORTANT** : une fois le TP terminé, penser à quitter l'instance JupyterHub pour **libérer le GPU** ( *> Hub Control Panel > Cancel* ).

### Optionnel : Profiler la validation

**TODO** : dans le script [dlojz1_4.py](./dlojz1_4.py) :
  * Ajouter des balises comme suit :
```python
    # Runs the forward pass with no grad mode.
    with torch.no_grad():
        # Implement autocasting
        with autocast():
            with record_function("inference"): val_outputs = model(val_images)
            with record_function("loss function"): val_loss = criterion(val_outputs, val_labels)
```
  * Enlever l'indication de fin d'itération d'apprentissage.
  * A la place, indiquer au *profiler* la fin de chaque itération de validation (dans la boucle de validation).
  * Reprendre une trace profiler
  * La visualiser sur Tensorboard

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.

Puis, rebasculer la cellule précédente en mode `Raw NBConvert`, afin d'eviter de relancer un job par erreur.

In [ ]:
#jobid = ['159708']

In [ ]:
display_slurm_queue(name)

In [ ]:
controle_technique(jobid)

**TODO** : dans le script [dlojz1_4.py](./dlojz1_4.py) :
* Vous pouvez ensuite changer la valeur de la variable `VAL_BATCH_SIZE=250`
* Reprendre une trace profiler
* La visualiser sur Tensorboard

![Garage](images/stop.png "Arrêtez-vous ici! Une présentation vous attend avant le prochain TP.")